### 1.0 Convolutional Neural Network

* convolutional layer
* relu layer
* pooling layer

In [123]:
# pytorch
import torch
from torch import nn
from torch.utils.data import DataLoader

# torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# machine learning
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt

# helper functions
from pathlib import Path
import requests
from tqdm.auto import tqdm
from timeit import default_timer as timer


# device
device = "cpu"

# print(torch.__version__)
# print(np.__version__)
# print(torchvision.__version__)

#### 2.0 Setup

* prepare & load data
* import helper functions
* create batches using data_loader
* automate train, test, eval functions

In [124]:
# PREPARE & LOAD DATA
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

# DECLARE ESSENTIAL INFO
class_names = train_data.classes
class_to_idx = train_data.class_to_idx

"""
print(class_names)
print(class_to_idx)
print(train_data, "\n", test_data)
"""

# SET BATCHES
BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

"""
print(train_dataloader, "\n", test_dataloader)
"""

# IMPORT HELPER FUNCTIONS
if Path("helper_functions.py").is_file():
    print("helper_functions.py already exists, skipping download...")
else:
    print("Downloading helper_functions.py")
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open("helper_functions.py", "wb") as f:
        f.write(request.content)

from helper_functions import accuracy_fn

def print_train_time(start: float,
                     end: float,
                     device: torch.device=None):
    """
    Prints difference between start adn end time.
    """

    total_time = end - start
    print(f"Train time on {device} : {total_time: .3f} seconds")
    return total_time


# AUTOMIZE TRAIN/TEST/EVAL FUNCTIONS
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device=device):
    """
    Trains model based on train data from data_loader.
    """

    model.train()
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):
        # put data on target device
        X, y = X.to(device), y.to(device)

        # make predictions
        y_pred = model.forward(X)

        # calculate & accumulate loss (wrongness)
        loss = loss_fn(y_pred, y)
        train_loss += loss 

        # calculate & accumulate acc
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # change logits -> prediction labels

        # optimize model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # keep track of progress
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)} samples.")
    

    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.2f}%")


def test_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn,
               device: torch.device=device):
    """
    Performs a test loop with model going over
    test data from data_loader.
    """

    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)

            test_pred = model.forward(X)
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        test_loss /= len(data_loader)
        test_acc /= len(data_loader)

    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%\n")


def eval_model(model: torch.nn.Module,
            data_loader: torch.utils.data.DataLoader,
            loss_fn: torch.nn.Module,
            accuracy_fn):
    """
    Returns a dictionary containing the results of model.
    Built for easy comparison between different models.
    The core function is equivalent to that of the "test_step" function.
    """

    model.eval()
    loss, acc = 0, 0

    with torch.inference_mode():
        for X, y in tqdm(data_loader):
            # make predictions
            y_pred = model.forward(X)

            # accumulate loss & acc per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y,
                                y_pred=y_pred.argmax(dim=1))
            
        # scale loss and acc to find average per batch
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__,
            "model_loss": loss.item(), 
            "model_acc": acc}

helper_functions.py already exists, skipping download...


#### 3.0 CNN Model

In [125]:
class FashionMNISTModelCNN(nn.Module):
    """
    Model architecture taht replicates TinyVGG model.
    """
    def __init__(self,
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        super().__init__()

        self.conv_block_1 = nn.Sequential(
                        nn.Conv2d(in_channels=input_shape,
                                  out_channels=hidden_units,
                                  kernel_size=3,
                                  stride=1,
                                  padding=1),
                        nn.ReLU(),
                        nn.Conv2d(in_channels=hidden_units,
                                  out_channels=hidden_units,
                                  kernel_size=3,
                                  stride=1,
                                  padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2)
        )

        self.conv_block_2 = nn.Sequential(
                        nn.Conv2d(in_channels=hidden_units,
                                  out_channels=hidden_units,
                                  kernel_size=3,
                                  stride=1,
                                  padding=1),
                        nn.ReLU(),
                        nn.Conv2d(in_channels=hidden_units,
                                  out_channels=hidden_units,
                                  kernel_size=3,
                                  stride=1,
                                  padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2) 
        )

        self.classifier = nn.Sequential(
                        nn.Flatten(),
                        nn.Linear(in_features=hidden_units*7*7,
                        out_features=output_shape)
        )
    
    # print shape to calculate in_features for nn.Linear()
    def forward(self, x):
        x = self.conv_block_1(x)
        print(f"Output shape of conv_block_1: {x.shape}")
        x = self.conv_block_2(x)
        print(f"Output shape of conv_block_2: {x.shape}")
        x = self.classifier(x)

        return x

In [126]:
torch.manual_seed(42)
model_cnn_1 = FashionMNISTModelCNN(input_shape=1,
                                   hidden_units=10,
                                   output_shape=len(class_names)).to(device)

In [127]:
# FIND IN_FEATURES THROUGH PRINTING SHAPES
rand_image_tensor = torch.randn(size=(1, 28, 28))

"""
# model_cnn_1(rand_image_tensor.unsqueeze(dim=0))

rand_image_tensor = torch.randn(size=(1, 10, 7, 7))
print(rand_image_tensor.size())

flatten_tensor = nn.Flatten()
rand_image_flattened = flatten_tensor(rand_image_tensor)
print(rand_image_flattened.size())

# print(f"Original image shape: {rand_image_tensor.shape}")
# print(f"Unsqueezed image shape: {rand_image_tensor.unsqueeze(dim=0).shape}")
"""

model_cnn_1(rand_image_tensor.unsqueeze(dim=0))

Output shape of conv_block_1: torch.Size([1, 10, 14, 14])
Output shape of conv_block_2: torch.Size([1, 10, 7, 7])


tensor([[ 0.0366, -0.0940,  0.0686, -0.0485,  0.0068,  0.0290,  0.0132,  0.0084,
         -0.0030, -0.0185]], grad_fn=<AddmmBackward0>)

#### 3.1 Exploring `nn.Conv2d()`

In [128]:
# create dummy data
torch.manual_seed(42)

images = torch.randn(size=(32, 3, 64, 64))
test_image = images[0]

"""
print(f"Image batch shape: {images.shape}")
print(f"Single image shape: {test_image.shape}")
print(f"Test image: \n {test_image}")
"""

'\nprint(f"Image batch shape: {images.shape}")\nprint(f"Single image shape: {test_image.shape}")\nprint(f"Test image: \n {test_image}")\n'

In [129]:
# model_cnn_1.state_dict()

In [130]:
# create single conv2d layer
torch.manual_seed(42)
conv_layer = nn.Conv2d(in_channels=3,
                       out_channels=10,
                       kernel_size=(3,3),
                       stride=1,
                       padding=1)

test_image = test_image.unsqueeze(dim=0)
conv_output = conv_layer(test_image)
print(test_image.shape, "\n", conv_output.shape)

torch.Size([1, 3, 64, 64]) 
 torch.Size([1, 10, 64, 64])


#### 3.2 Exploring `nn.nn.MaxPool2d()`

In [131]:
test_image = images[1]

print(f"Test image original shape: {test_image.shape}")
print(f"Test image with unsqueezed dimension: {test_image.unsqueeze(dim=0).shape}")

max_pool_layer = nn.MaxPool2d(kernel_size=2)

test_image_through_conv = conv_layer(test_image.unsqueeze(dim=0))
print(f"Shape after going through conv layer: {test_image_through_conv.shape}")

test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
print(f"Shape after going through conv and max pool layer: {test_image_through_conv_and_max_pool.shape}")

Test image original shape: torch.Size([3, 64, 64])
Test image with unsqueezed dimension: torch.Size([1, 3, 64, 64])
Shape after going through conv layer: torch.Size([1, 10, 64, 64])
Shape after going through conv and max pool layer: torch.Size([1, 10, 32, 32])
